In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam

from scipy.stats import spearmanr
import torch 

import os
from collections import Counter
from helpers import *

1. Load the dataset 

In [ ]:
#load training data (will be put in a function later)  
path = os.getcwd()
for i in range(3) :

    path = os.path.dirname(path)

path += '/data/'
train_df = pd.read_csv(path + 'train_v1.csv',index_col="seq_id")
train_df = train_df.drop(columns=['data_source'])
train_df = train_df.dropna()
train_df.head()

2. Translate Amino-acids to numbers and create a One-Channel array for each sequence

In [ ]:
# Count the frequency of each amino acid in the training set
map_dict = {} 
def count_frequency(s):
    counter = Counter(s)
    for p in s  : 
        if p in map_dict : 
            map_dict[p]+=counter[p]
        else : 
            map_dict[p]= counter[p]
        

In [ ]:
# fill the map_dict with the frequency of each amino acid  (TODO:  better code )
_ = train_df.apply(lambda row : count_frequency(row['protein_sequence']),axis=1)
map_dict = dict(sorted(map_dict.items(), key=lambda item: item[1],reverse=True))

# Assign a number to each amino acid based on its frequency in decreasing order
i = 1
for key, value in map_dict.items():
    map_dict[key]= i
    i+=1

In [ ]:
#map each amino acid to its number
def chr_to_int(s):
    l = []
    for ch in s :
       l.append(map_dict[ch])
    return l


In [ ]:
#create a new column in the dataframe with the  amino acids in numerical form (array of numbers from 1 to 20)
train_df['numerical_sequence'] = train_df.apply(lambda row : chr_to_int(row['protein_sequence']),axis=1)

In [ ]:
#add a new column that contains the length of each protein sequence (before padding)
train_df['length'] = train_df['protein_sequence'].str.len()

max_length=np.max(train_df['length'])

In [ ]:
#final dataframe 
train_df.head()

**3. Padding with zeros**

In [ ]:
## add 0 to numerical_sequence to make all of them the same length
padded_train_df = train_df.copy()
padded_train_df['numerical_sequence'] = train_df.apply(lambda row : row['numerical_sequence'] + [0]*(max_length - row['length']),axis=1)

In [ ]:
#Prepare dataframe to Pass to the dataloader (will be put in a function later)
padded_train_df.drop(columns=['protein_sequence'],inplace=True)
padded_train_df['y'] = padded_train_df['tm']
padded_train_df.drop(columns=['tm'],inplace=True)
padded_train_df['numerical_sequence'] = padded_train_df.apply(lambda row : np.array(row['numerical_sequence']),axis=1)

In [ ]:
df = padded_train_df.copy()

# Convolutional NN + MLP

number of channels : 1 <br>

In [ ]:
# hyperparameters
batch_size = 128
learning_rate = 0.001 # Suggested for Adam
num_epochs = 10

In [ ]:
class EnzymesDataset(Dataset):
 
  def __init__(self,df):
    
    # the Amino acid sequences as an int array
    sequence= df.iloc[:]['numerical_sequence']
    # numerical : pH and length
    numerical = df.iloc[:,[0,2]].values

    # y : the target (tm)
    y=df.iloc[:,3].values
  
    #create tensors from the numpy arrays
    self.x_sequence=torch.tensor(sequence)
    self.y=torch.tensor(y,dtype=torch.float32)
    self.num=torch.tensor(numerical,dtype=torch.float32)
 
  def __len__(self):
    return len(self.y)
   
  def __getitem__(self,idx):
    return self.x_sequence[idx],self.y[idx] , self.num[idx]


In [ ]:
class Conv1D_OneChannel(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.protein_sequence = nn.Sequential(
            nn.Conv1d(1, 1,kernel_size=8, stride=1),
            nn.ReLU(),
            nn.Dropout(),


        )

        self.prot_seq_one_pooling = nn.Sequential(

            #With pooling only at the end (seen in paper)

            nn.Conv1d(1, 64,kernel_size=8, stride=1),
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv1d(64, 64, 5, stride=1, padding=2), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Conv1d(64, 64, 5, stride=1, padding=2), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Conv1d(64, 64, 5, stride=1, padding=2), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Conv1d(64, 64, 5, stride=1, padding=2), 
            nn.ReLU(), 
            nn.Dropout(),
            nn.Conv1d(64, 32, 5, stride=1, padding=1), 
            nn.ReLU(), 
            nn.Dropout(),
            #nn.AdaptiveAvgPool1d(32), #argument = output size 
            nn.Conv1d(32, 1, 5, stride=1, padding=1), 
            nn.ReLU(), 
            nn.Dropout(),


        )
        self.numerical = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(64, 64),
            nn.ReLU(),
        )
        self.mlp = nn.Sequential(
            nn.Linear(94, 64),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(64, 1),
        )

    def forward(self, x,y):
        x = self.prot_seq_one_pooling(x.float())
        y = self.numerical(y)
       
        x = torch.cat((x.squeeze(1), y), 1)
        x = self.mlp(x)
        return x

**3. Cross validation**

In [ ]:
from sklearn.model_selection import KFold

#CROSS VALIDATION 

k_folds = 5
learning_rate = 1e-4
kfold = KFold(n_splits=k_folds, shuffle=True)
dataset = EnzymesDataset(df.reset_index(drop=True))
train_loss_history = []
test_loss_history = []
train_rho_history = []
test_rho_history = []
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    train_dl = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=32, sampler=train_subsampler)
    val_dl = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=32, sampler=test_subsampler)

    model = Conv1D_OneChannel()
    optimizer = Adam(model.parameters(), lr=learning_rate)
    # defining the loss function
    criterion = nn.MSELoss()
    # checking if GPU is available
    if torch.cuda.is_available():
        model = model.cuda()
        criterion = criterion.cuda()
    
    for epoch in range(1, num_epochs + 1):
        train_loss , rho_train = train_epoch( model, optimizer, criterion, train_dl, epoch)
    
    
    test_loss , rho_test = test_epoch(model, criterion, val_dl)
        
    train_loss_history.append(train_loss)
    train_rho_history.append(rho_train)
    test_loss_history.append(test_loss)
    test_rho_history.append(rho_test)

    print(f'for fold {fold} : \n train_loss :  {train_loss}     test_loss : {test_loss} \n \n')
    
    
    
 


**4. Train model and plot**

In [ ]:

train_df , val_df = split_train_test(df,frac=0.8)

In [ ]:
# create pytorch datasets
train_d = EnzymesDataset(train_df)
val_d = EnzymesDataset(val_df)


# create pytorch dataloaders
train_dl = torch.utils.data.DataLoader(train_d, batch_size=batch_size, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_d, batch_size=batch_size, shuffle=True)

In [ ]:

model = Conv1D_OneChannel()
optimizer = Adam(model.parameters(), lr=learning_rate)
# defining the loss function
criterion = nn.MSELoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()


In [ ]:
# train and test the model (save it after each epoch)
train_loss_history = []
test_loss_history = []
train_rho_history = []
test_rho_history = []
for epoch in range(1, num_epochs + 1):
    train_loss , rho_train = train_epoch(
        model, optimizer, criterion, train_dl, epoch
    )
    train_loss_history.append(train_loss)
    train_rho_history.append(rho_train)
    
    test_loss , rho_test = test_epoch(model, criterion, val_dl)
    test_loss_history.append(test_loss)
    test_rho_history.append(rho_test)
    
    
#torch.save(model.state_dict(), f"1-Conv1d_model.pth")


In [ ]:
#create loss plot
plt.plot(train_loss_history, label='train loss')
plt.plot(test_loss_history, label='test loss')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title(' train and test MSE Loss')
plt.legend()
plt.savefig('Rendu1-conv1d_Loss.png')


In [ ]:
plt.plot(train_rho_history, label='train rho')
plt.plot(test_rho_history, label='test rho')
plt.xlabel('Epoch')
plt.ylabel('rho')
plt.title(' Spearman\'s rank correlation coefficient')
plt.legend()
plt.savefig('Rendu1-conv1d_rho.png')